# In this script you will calculate MAG relative abundance by normalizing mAG Truncated Average sequencing Depth (TAD80) values and metagenome Genome Equivalent values. You will also use metagenome size (number of reads and number of base pairs) to calculate other coverage statistics.

## Input must include the following column-separated files
### 1. List of MAGs and corresponding sizes (two columns)
### 2. List of metagenomes and corresponding number of reads and number of base pairs, as separate columns. In my sample file I also have column 2 containing sample metadata ('Day').
### 3. List of MAGs to check against #1
### 4. MAG-specific files containing raw TAD80 values for each sample (MAGs in column one, TAD80s in column two). These should be in a subdirectory, which I call here "tads_RAW".
### 5. Sample-specific files containing the Microbe Census output, including Genome Equivalent value

In [1]:
import numpy as np
import pandas as pd
from functools import reduce
import os as os
import glob as glob

In [2]:
insert your own working directory here
os.chdir("/Users/npatin3/Dropbox (GaTech)/Norovirus/TADs/Jupyter/Data")

### Constant dataframes

In [3]:
MAG_list = pd.read_csv("Noro_dRep_winners.csv", names=['MAG'])
MAG_sizes = pd.read_csv("MAG_sizes.csv", header=0, names=['MAG','Size'])
mgn_bps = pd.read_csv("Noro_mgn_reads.csv")

In [5]:
#sanity check
MAG_list.head()

,MAG
0,13_1_001
1,14_1_002
2,14_1_003
3,14_1_005
4,14_4_005


### Check if there are any MAGs from the list that didn't end up in the merged df with sizes

In [118]:
MAG_list[(~MAG_list.MAG.isin(sizes.MAG))]

### Function to extract Genome Equivalents value from text file

In [6]:
def extract_GE(GE):
    """Use the MicrobeCensus output file to extract the Genome Equivalent value of the metagenome"""
    with open(GE, "rt") as GE_file:
        # read all lines into a list
        lines = GE_file.readlines()
        # extract just the genome equivalent value, which is the second value on the 12th line
        ge = float(lines[12].split('\t')[1])
    return(ge)

### Loop through all MAG-specific files with raw TAD values and use each file to build new dataframe with sample and coverage data

In [7]:
df = []
for file in glob.glob("tads_RAW/*.csv"):
    a, b = file.split('/')
    c, d = b.split('.')
    e, f = c.split('-')
    raw_tads = pd.read_csv(file, names=['MAG','Raw_TAD'])
    # Merge all three dfs: MAG list, MAG sizes, and raw TADs
    dfs = [MAG_list, raw_tads, MAG_sizes]
    # make new data frame 'tads'
    tads = reduce(lambda left,right: pd.merge(left, right, on=['MAG']), dfs)
    # extract GE for sample
    GE_path = '%s_GE.txt' % e
    x = extract_GE(GE_path)
    # make GE list for each MAG
    ge_list = x * len(tads.index)
    bps = mgn_bps.loc[mgn_bps['Sample'] == e, 'bp'].iloc[0]
    bps_list = bps * len(tads.index)
    # make new dataframe 'tads' and populate it with all the stats
    tads['sample'] = e
    tads['GE'] = ge_list
    tads['mgn_bps'] = bps_list
    tads['Normalized_coverage'] = tads['Raw_TAD'] / tads['GE']
    tads['MAG_size'] = MAG_sizes['Size']
    tads['MAG_proportion'] = (tads['Raw_TAD'] * tads['MAG_size']) / tads['mgn_bps']
    tads = tads[['sample','GE','mgn_bps','MAG','Raw_TAD','MAG_size','Normalized_coverage','MAG_proportion']]
    #print(file, b, e)
    df.append(tads)
    
total_df = pd.concat(df)

In [8]:
total_df.head()

,sample,GE,mgn_bps,MAG,Raw_TAD,MAG_size,Normalized_coverage,MAG_proportion
0,13_1,36751.376735,74426278660,13_1_001,125.673472,4374699,0.003420,0.007387
1,13_1,36751.376735,74426278660,14_1_002,0.000000,2251753,0.000000,0.000000
2,13_1,36751.376735,74426278660,14_1_003,0.388714,4356764,0.000011,0.000023
3,13_1,36751.376735,74426278660,14_1_005,0.000000,2787927,0.000000,0.000000
4,13_1,36751.376735,74426278660,14_4_005,0.000000,2878704,0.000000,0.000000


In [9]:
total_df.shape

(1742, 8)

#### Reformat data frame to show just values of interest, eg TAD80 values ('Normalized_coverage')

In [12]:
df_for_table = pd.pivot_table(data=df_to_save, index='MAG', values='Normalized_coverage', columns='sample')

In [13]:
df_for_table.head()

sample,13_1,13_2,13_4,15_1,15_3,15_6,15_7,15_8,15_9,28_1,...,37_8,37_9,38_1,38_6,41_1,41_7,49_1,49_3,49_4,4_1
MAG,,,,,,,,,,,,,,,,,,,,,
13_1_001,0.003420,0.004224,0.002117,0.000270,0.000110,2.527719e-04,0.000472,0.000139,0.000385,0.000052,...,0.000489,0.000416,0.000935,0.001040,0.001795,0.002725,8.062754e-04,0.000227,0.000882,0.001432
14_1_002,0.000000,0.000000,0.000000,0.000006,0.000002,1.531116e-05,0.000012,0.000008,0.000013,0.000068,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
14_1_003,0.000011,0.000000,0.000002,0.000000,0.000000,2.337960e-07,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000005,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
14_1_005,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000164,...,0.000000,0.000000,0.000000,0.000000,0.000275,0.000160,1.041480e-06,0.000000,0.000000,0.000000
14_4_005,0.000000,0.000000,0.000000,0.000081,0.000040,9.936179e-05,0.000033,0.000023,0.000011,0.000000,...,0.000060,0.000055,0.000000,0.000000,0.000238,0.000076,7.173552e-07,0.000000,0.000000,0.000000


In [14]:
df_for_table.to_csv("MAG_TAD80s.csv")